# Importing and Exploring Data
https://cccbdb.nist.gov/exp1x.asp

In [4]:
# store the data obtained from cccbdb
H2_geo = {
    'H1': [0.0, 0.0, 0.0],
    'H2': [0.0, 0.0, 0.7414]
}
H2O_geo = {
    'O1': [0.0, 0.0, 0.1173],
    'H2': [0.0, 0.7572, -0.4692],
    'H3': [0.0, -0.7572, -0.4692]
}
benzene_geo = {
    'C1': [0.0000, 1.3970, 0.0000],
    'C2': [1.2098, 0.6985, 0.0000],
    'C3': [1.2098, -0.6985, 0.0000],
    'C4': [0.0000, -1.3970, 0.0000],
    'C5': [-1.2098, -0.6985, 0.0000],
    'C6': [-1.2098, 0.6985, 0.0000],
    'H7': [0.0000, 2.4810, 0.0000],
    'H8': [2.1486, 1.2405, 0.0000],
    'H9': [2.1486, -1.2405, 0.0000],
    'H10': [0.0000, -2.4810, 0.0000],
    'H11': [-2.1486, -1.2405, 0.0000],
    'H12': [-2.1486, 1.2405, 0.0000]
}

In [5]:
H2_geo

{'H1': [0.0, 0.0, 0.0], 'H2': [0.0, 0.0, 0.7414]}

In [6]:
H2O_geo

{'O1': [0.0, 0.0, 0.1173],
 'H2': [0.0, 0.7572, -0.4692],
 'H3': [0.0, -0.7572, -0.4692]}

In [7]:
benzene_geo

{'C1': [0.0, 1.397, 0.0],
 'C2': [1.2098, 0.6985, 0.0],
 'C3': [1.2098, -0.6985, 0.0],
 'C4': [0.0, -1.397, 0.0],
 'C5': [-1.2098, -0.6985, 0.0],
 'C6': [-1.2098, 0.6985, 0.0],
 'H7': [0.0, 2.481, 0.0],
 'H8': [2.1486, 1.2405, 0.0],
 'H9': [2.1486, -1.2405, 0.0],
 'H10': [0.0, -2.481, 0.0],
 'H11': [-2.1486, -1.2405, 0.0],
 'H12': [-2.1486, 1.2405, 0.0]}

# Bond Length Calculation

In [21]:
import math
import warnings
def compute_bond_length(coord1, coord2):
    """
    Computes the bond length between two atoms.

    Parameters:
        coord1: 1D array-like, cartesian coordinates of the first atom in angstroms
        coord2: 1D array-like, cartesian coordinates of the second atom in angstroms, must be of the same length as coord1
        
    Returns:
        The Euclidean distance between coord1 and coord2
    """
    d = math.sqrt(sum([(comp1-comp2)**2 for comp1, comp2 in zip(coord1, coord2)]))
    if (d >= 2):
        warnings.warn(f"The bond length of {d} angstroms is atypical for a covalent bond.")
    return d

In [22]:
# test
print(compute_bond_length(H2O_geo['O1'], H2O_geo['H2']))
print(compute_bond_length([0, 0], [1, 1])) # should be sqrt(2)
print(compute_bond_length([0, 0, 0], [3, 3, 3])) # should print warning

0.9577755948028744
1.4142135623730951
5.196152422706632


C:\Users\repti\AppData\Local\Temp\ipykernel_4216\233551919.py:16: UserWarning: The bond length of 5.196152422706632 angstroms is atypical for a covalent bond.
  warnings.warn(f"The bond length of {d} angstroms is atypical for a covalent bond.")


# Bond Angle Calculation
*Note*: I am using the formula
$$ cos\theta = \frac{\vec{BA} \cdot \vec{BC}} {|\vec{BA}||\vec{BC}|} $$
in calculating the angle using B as the vertex, which gives consistent results with regard to whether the angle is acute. The formula given in the instructions yields 180 subtracting what is visually perceived as the angle.

In [55]:
import numpy as np
def compute_bond_angle(coord1, coord2, coord3):
    """
    Computes the bond angle between three atoms.
    coord1 -- coord2
                   |_\\
                      \\
                        coord3

    Parameters:
        All parameters must be of the same dimensions
        coord1: 1D array-like, cartesian coordinates of the first atom in angstroms
        coord2: 1D array-like, cartesian coordinates of the second atom in angstroms
        coord3: 1D array-like, cartesian coordinates of the third atom in angstroms
    Returns:
        The Euclidean distance between coord1 and coord2
    """
    # put into arrays
    A = np.array(coord1)
    B = np.array(coord2)
    C = np.array(coord3)
    # calculate vectors
    vec1 = A - B
    vec2 = C - B
    
    numer = np.dot(vec1, vec2) # dot product
    denom = np.linalg.norm(vec1) * np.linalg.norm(vec2) # norms multiplied
    theta_rad = math.acos(numer / denom) # get theta in radians
    theta_deg = theta_rad * 180 / math.pi # convert to degrees
    if theta_deg < 90:
        print(f"Acute angle of {theta_deg:.2f}")
    elif theta_deg > 90:
        print(f"Obtuse angle of {theta_deg:.2f}")
    else:
        print(f"Right angle of {theta_deg:.2f}")
    return theta_deg

In [60]:
# tests
compute_bond_angle([0,0,0], [1,0,0], [1,1,0]) # should be right angle
compute_bond_angle([0,0], [1,0], [0.5, 1]) # should be acute
compute_bond_angle([0,0], [1,0], [3,1]); # should be obtuse

Right angle of 90.00
Acute angle of 63.43
Obtuse angle of 153.43


# Automating the Calculation of Unique Bond Lengths and Angles

## Calculating all unique bond lengths

In [68]:
def calculate_all_bond_lengths(molecule):
    """
    Calculates all unique bond lengths

    Parameters:
        molecule: a dictionary with atom names as keys and 1D array-like Cartesian coordinates as values

    Returns:
        All unique bond lengths in a list
    """

    keys = list(molecule.keys())
    bond_lengths = []
    for i in range(0, len(keys)): # loop over all atoms
        atom1 = keys[i]
        coord1 = molecule[atom1] # get the first coordinates
        for j in range(i+1, len(keys)): # loop over all atoms after the ith one
            atom2 = keys[j]
            coord2 = molecule[atom2] # get the second coordinates
            bond_length = compute_bond_length(coord1, coord2) # compute bond length
            bond_lengths.append(bond_length) # store in list
    print(f"All unique bond lengths: {bond_lengths}")
    return bond_lengths

In [73]:
# tests
calculate_all_bond_lengths(H2_geo)
calculate_all_bond_lengths(H2O_geo)
calculate_all_bond_lengths(benzene_geo);

All unique bond lengths: [0.7414]
All unique bond lengths: [0.9577755948028744, 0.9577755948028744, 1.5144]
All unique bond lengths: [1.3969675336241714, 2.4196562338481056, 2.794, 2.4196562338481056, 1.3969675336241714, 1.0839999999999999, 2.1542920438046465, 3.4018947970212134, 3.878, 3.4018947970212134, 2.1542920438046465, 1.397, 2.4196562338481056, 2.7939350672483427, 2.4196, 2.1542799934084704, 1.0840246491662446, 2.154313449802512, 3.401887165971264, 3.8779597161394035, 3.401854576550855, 1.3969675336241714, 2.4196, 2.7939350672483427, 3.401887165971264, 2.154313449802512, 1.0840246491662446, 2.1542799934084704, 3.401854576550855, 3.8779597161394035, 1.3969675336241714, 2.4196562338481056, 3.878, 3.4018947970212134, 2.1542920438046465, 1.0839999999999999, 2.1542920438046465, 3.4018947970212134, 1.397, 3.401887165971264, 3.8779597161394035, 3.401854576550855, 2.1542799934084704, 1.0840246491662446, 2.154313449802512, 2.1542799934084704, 3.401854576550855, 3.8779597161394035, 3.401

C:\Users\repti\AppData\Local\Temp\ipykernel_4216\233551919.py:16: UserWarning: The bond length of 2.4196562338481056 angstroms is atypical for a covalent bond.
  warnings.warn(f"The bond length of {d} angstroms is atypical for a covalent bond.")
C:\Users\repti\AppData\Local\Temp\ipykernel_4216\233551919.py:16: UserWarning: The bond length of 2.794 angstroms is atypical for a covalent bond.
  warnings.warn(f"The bond length of {d} angstroms is atypical for a covalent bond.")
C:\Users\repti\AppData\Local\Temp\ipykernel_4216\233551919.py:16: UserWarning: The bond length of 2.1542920438046465 angstroms is atypical for a covalent bond.
  warnings.warn(f"The bond length of {d} angstroms is atypical for a covalent bond.")
C:\Users\repti\AppData\Local\Temp\ipykernel_4216\233551919.py:16: UserWarning: The bond length of 3.4018947970212134 angstroms is atypical for a covalent bond.
  warnings.warn(f"The bond length of {d} angstroms is atypical for a covalent bond.")
C:\Users\repti\AppData\Local\

In [74]:
len(calculate_all_bond_lengths(benzene_geo)) # should be 12 choose 2 = 12*11/2 = 66

All unique bond lengths: [1.3969675336241714, 2.4196562338481056, 2.794, 2.4196562338481056, 1.3969675336241714, 1.0839999999999999, 2.1542920438046465, 3.4018947970212134, 3.878, 3.4018947970212134, 2.1542920438046465, 1.397, 2.4196562338481056, 2.7939350672483427, 2.4196, 2.1542799934084704, 1.0840246491662446, 2.154313449802512, 3.401887165971264, 3.8779597161394035, 3.401854576550855, 1.3969675336241714, 2.4196, 2.7939350672483427, 3.401887165971264, 2.154313449802512, 1.0840246491662446, 2.1542799934084704, 3.401854576550855, 3.8779597161394035, 1.3969675336241714, 2.4196562338481056, 3.878, 3.4018947970212134, 2.1542920438046465, 1.0839999999999999, 2.1542920438046465, 3.4018947970212134, 1.397, 3.401887165971264, 3.8779597161394035, 3.401854576550855, 2.1542799934084704, 1.0840246491662446, 2.154313449802512, 2.1542799934084704, 3.401854576550855, 3.8779597161394035, 3.401887165971264, 2.154313449802512, 1.0840246491662446, 2.4809921825753505, 4.297213540190899, 4.962, 4.2972135

C:\Users\repti\AppData\Local\Temp\ipykernel_4216\233551919.py:16: UserWarning: The bond length of 2.4196562338481056 angstroms is atypical for a covalent bond.
  warnings.warn(f"The bond length of {d} angstroms is atypical for a covalent bond.")
C:\Users\repti\AppData\Local\Temp\ipykernel_4216\233551919.py:16: UserWarning: The bond length of 2.794 angstroms is atypical for a covalent bond.
  warnings.warn(f"The bond length of {d} angstroms is atypical for a covalent bond.")
C:\Users\repti\AppData\Local\Temp\ipykernel_4216\233551919.py:16: UserWarning: The bond length of 2.1542920438046465 angstroms is atypical for a covalent bond.
  warnings.warn(f"The bond length of {d} angstroms is atypical for a covalent bond.")
C:\Users\repti\AppData\Local\Temp\ipykernel_4216\233551919.py:16: UserWarning: The bond length of 3.4018947970212134 angstroms is atypical for a covalent bond.
  warnings.warn(f"The bond length of {d} angstroms is atypical for a covalent bond.")
C:\Users\repti\AppData\Local\

66

## Calculating all unique bond angles

In [123]:
def calculate_all_bond_angles(molecule, ignore_order=True, debug=False):
    """
    Calculates all unique bond angles

    Parameters:
        molecule: a dictionary with atom names as keys and 1D array-like Cartesian coordinates as values
        ignore_order = True: if set to true, treat each three atoms consituting an angle as an unordered set and skip duplicate sets. If set to false, treat as an ordered set.
        debug = False: sets debug mode, which will print the angle sequence along with its magnitude in degrees

    Returns:
        All unique bond angles in a list
    """

    calculated = set() # record of already calculated pairs to avoid duplicates
    bond_angles = []
    for atom1, coord1 in molecule.items():
        for atom2, coord2 in molecule.items():
            if (atom1 == atom2):
                continue # skip self calculation
            for atom3, coord3 in molecule.items():
                if atom1 == atom3 or atom2 == atom3:
                    continue # skip self calculation
                if ignore_order:
                    this_angle = tuple(sorted([atom1, atom2, atom3])) # sorted so that same atoms produce the same sequence; cast to tuple so that it is hashable to store in a set
                else:
                    this_angle = (atom1, atom2, atom3) # different order considered non-duplicates
                if this_angle in calculated: # if already recorded
                    continue
                
                if debug:
                    print(this_angle)
                calculated.add(this_angle) # record this sequence
                bond_angle = compute_bond_angle(coord1, coord2, coord3) # calculate bond angle
                bond_angles.append(bond_angle) # store in list
    
    print(f"All unique bond angles: {bond_angles}")
    return bond_angles

In [124]:
# tests
calculate_all_bond_angles(H2O_geo, ignore_order=False, debug=True);

('O1', 'H2', 'H3')
Acute angle of 37.76
('O1', 'H3', 'H2')
Acute angle of 37.76
('H2', 'O1', 'H3')
Obtuse angle of 104.48
('H2', 'H3', 'O1')
Acute angle of 37.76
('H3', 'O1', 'H2')
Obtuse angle of 104.48
('H3', 'H2', 'O1')
Acute angle of 37.76
All unique bond angles: [37.76008059081679, 37.76008059081679, 104.47983881836642, 37.76008059081679, 104.47983881836642, 37.76008059081679]


In [125]:
calculate_all_bond_angles(H2_geo)
calculate_all_bond_angles(benzene_geo)

All unique bond angles: []
Obtuse angle of 120.00
Obtuse angle of 90.00
Acute angle of 60.00
Acute angle of 30.00
Acute angle of 25.83
Obtuse angle of 120.00
Obtuse angle of 145.84
Obtuse angle of 99.17
Acute angle of 60.00
Acute angle of 20.83
Obtuse angle of 90.00
Acute angle of 60.00
Acute angle of 30.00
Acute angle of 9.17
Acute angle of 55.83
Obtuse angle of 150.00
Obtuse angle of 115.84
Acute angle of 69.17
Acute angle of 30.00
Acute angle of 60.00
Acute angle of 30.00
Acute angle of 0.00
Acute angle of 39.17
Acute angle of 85.83
Obtuse angle of 180.00
Acute angle of 85.83
Acute angle of 39.17
Acute angle of 30.00
Acute angle of 9.17
Acute angle of 30.00
Acute angle of 69.17
Obtuse angle of 115.84
Obtuse angle of 150.00
Acute angle of 55.83
Acute angle of 25.83
Acute angle of 20.83
Acute angle of 60.00
Obtuse angle of 99.17
Obtuse angle of 145.84
Obtuse angle of 120.00
Acute angle of 60.00
Acute angle of 30.00
Acute angle of 0.00
Acute angle of 30.00
Acute angle of 60.00
Obtuse a

[120.00076879612567,
 90.0015375803399,
 60.001537592251296,
 30.000768796125648,
 25.834113318202856,
 119.9999833867389,
 145.8355072470542,
 99.16894489628541,
 60.00111242571096,
 20.83305800159326,
 90.0015375803399,
 60.000768784214245,
 29.999999988088607,
 9.16740731594547,
 55.8339696667143,
 149.99847903292124,
 115.83565089854275,
 69.16847957874664,
 30.00042515462894,
 59.999231203874366,
 29.999231215785734,
 0.0,
 39.167468103359994,
 85.83403968417791,
 180.0,
 85.83403968417791,
 39.167468103359994,
 29.999231215785734,
 9.16740731594547,
 30.00042515462894,
 69.16847957874664,
 115.83565089854275,
 149.99847903292124,
 55.8339696667143,
 25.834113318202856,
 20.83305800159326,
 60.00111242571096,
 99.16894489628541,
 145.8355072470542,
 119.9999833867389,
 59.99989576812718,
 29.999895768346136,
 0.0,
 29.999895768346136,
 59.99989576812718,
 94.16596031582208,
 64.16606454747595,
 34.16606454769491,
 4.165960315822057,
 80.83263612851283,
 50.83253189664001,
 20.8324

In [130]:
len(calculate_all_bond_angles(benzene_geo)) # should be 12 choose 3 = 12*11*10/(3*2) = 220

Obtuse angle of 120.00
Obtuse angle of 90.00
Acute angle of 60.00
Acute angle of 30.00
Acute angle of 25.83
Obtuse angle of 120.00
Obtuse angle of 145.84
Obtuse angle of 99.17
Acute angle of 60.00
Acute angle of 20.83
Obtuse angle of 90.00
Acute angle of 60.00
Acute angle of 30.00
Acute angle of 9.17
Acute angle of 55.83
Obtuse angle of 150.00
Obtuse angle of 115.84
Acute angle of 69.17
Acute angle of 30.00
Acute angle of 60.00
Acute angle of 30.00
Acute angle of 0.00
Acute angle of 39.17
Acute angle of 85.83
Obtuse angle of 180.00
Acute angle of 85.83
Acute angle of 39.17
Acute angle of 30.00
Acute angle of 9.17
Acute angle of 30.00
Acute angle of 69.17
Obtuse angle of 115.84
Obtuse angle of 150.00
Acute angle of 55.83
Acute angle of 25.83
Acute angle of 20.83
Acute angle of 60.00
Obtuse angle of 99.17
Obtuse angle of 145.84
Obtuse angle of 120.00
Acute angle of 60.00
Acute angle of 30.00
Acute angle of 0.00
Acute angle of 30.00
Acute angle of 60.00
Obtuse angle of 94.17
Acute angle o

220

In [129]:
len(calculate_all_bond_angles(benzene_geo, ignore_order=False)) # should be 12 permutate 3 = 12*11*10 = 1320

Obtuse angle of 120.00
Obtuse angle of 90.00
Acute angle of 60.00
Acute angle of 30.00
Acute angle of 25.83
Obtuse angle of 120.00
Obtuse angle of 145.84
Obtuse angle of 99.17
Acute angle of 60.00
Acute angle of 20.83
Acute angle of 30.00
Obtuse angle of 90.00
Acute angle of 60.00
Acute angle of 30.00
Acute angle of 9.17
Acute angle of 55.83
Obtuse angle of 150.00
Obtuse angle of 115.84
Acute angle of 69.17
Acute angle of 30.00
Acute angle of 30.00
Acute angle of 60.00
Acute angle of 60.00
Acute angle of 30.00
Acute angle of 0.00
Acute angle of 39.17
Acute angle of 85.83
Obtuse angle of 180.00
Acute angle of 85.83
Acute angle of 39.17
Acute angle of 30.00
Acute angle of 60.00
Obtuse angle of 90.00
Acute angle of 30.00
Acute angle of 9.17
Acute angle of 30.00
Acute angle of 69.17
Obtuse angle of 115.84
Obtuse angle of 150.00
Acute angle of 55.83
Acute angle of 30.00
Acute angle of 60.00
Obtuse angle of 90.00
Obtuse angle of 120.00
Acute angle of 25.83
Acute angle of 20.83
Acute angle of

1320